# Only for Colab

In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
# %cd drive/MyDrive/pyFM

In [3]:
# !pip install potpourri3d robust_laplacian open3d trimesh tensorboard

Before this, run split_data_DA

In [4]:
import torch

In [5]:
torch.cuda.empty_cache()

# Train and evaluate dpfm model

In [6]:
import yaml
from dann.main import main
from dann.test import test_target, test_source
from project.datasets import Tosca
from project.train_dpfm import train_net
from project.eval_dpfm import eval_net
from dann.main_test import main_test

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [7]:
cfg = yaml.safe_load(open(f"dann/config/tosca_cuts_da.yaml", "r"))
source_domain_path = '../data/Tosca_DA/source/train'
target_domain_path = '../data/Tosca_DA/target/train'
source_test_path = '../data/Tosca_DA/source/test'
target_test_path = '../data/Tosca_DA/target/test'
torch.cuda.empty_cache()


In [8]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

# Clear any logs from previous runs
# !rm -rf ./runs/ 

In [9]:
%tensorboard --logdir runs 


Reusing TensorBoard on port 6006 (pid 13824), started 2 days, 0:08:24 ago. (Use '!kill 13824' to kill it.)

In [10]:
main(source_domain_path,target_domain_path,cfg)

using dataset cache path: ../data/Tosca_DA/source/train\cache_cuts_train.pt
  --> loading dataset from cache
using dataset cache path: ../data/Tosca_DA/target/train\cache_cuts_train.pt
  --> loading dataset from cache
Source-only training


100%|███████████████████████████████████████████████████████████████████████████████████| 1/1 [01:50<00:00, 110.51s/it]


#epoch:1, #iteration:89, train_loss:170.34587624367703, val_loss:8.818007510641348
DANN training


100%|███████████████████████████████████████████████████████████████████████████████████| 1/1 [02:35<00:00, 155.68s/it]

#epoch:1, train_loss:120.51419251688411, val_loss_src:8.680221764937691, val_loss_tgt:31.911810418833856


**Note**: Change model path to your respective path

### FM evaluation

In [ ]:
pred_p2p_list_tgt, pred_p2p_list_src = main_test(source_test_path, target_test_path, 'dann/data/dann_training.pth', cfg, 'test_tosca.pt', n_samples=None)


using dataset cache path: ../data/Tosca_DA/source/test\cache_cuts_train.pt
  --> loading dataset from cache
using dataset cache path: ../data/Tosca_DA/target/test\cache_cuts_train.pt
  --> loading dataset from cache
Starting evaluation...
Using device cuda:0
Loading model...
Starting inference...


  0%|                                                                                           | 0/44 [00:00<?, ?it/s]

In [ ]:
#p2p_fm, distances_fm = eval_net(cfg, "project/data/saved_models_cuts_3/ep_49.pth", "data/eval/test_tosca.pt", return_pred_p2p=True, return_dist=True, mode="FM")

### CFM evaluation

In [ ]:
#p2p_cfm, distances_cfm = eval_net(cfg,"project/data/saved_models_cuts_3/ep_49.pth", "data/eval/test_tosca.pt", return_pred_p2p=True, return_dist=True, mode="CFM")

# Plot results

# Notes
- Trained on 26 shapes, tested on 6 shapes (80/20 train/test split)
- Training process seems to work well, can't tell if it's overfitting due to lack of validation implementation
- Errors:
  - WKS: 5.99
  - DPFM trained on 26 shapes: 0.66
  - DPFM trained on 60% of TOSCA: 0.50
- Compared to FM with hand-crafted descriptors (WKS): 5.99 <-> 0.66 (mean normalized geodesic error)
- Improvements:
  - validation set implementation
  - tensorboard visualization
  - lr scheduler from pytorch
  - inference is slow, mostly done on CPU, transfer to GPU
- Next steps:
  - double check all dataloaders & FM to p2p conversions
  - training on whole partial-to-full dataset (TOSCA), or mix full-to-full & partial-to-full, e.g. warm start with model pre-trained on an easier problem, i.e. full-to-full
  - combine model w/ coupled FM
  - implement gradient clipping
  - apply trained descriptors to CFM


# Change log

- Remove data augmentation in validation step
- LR scheduler decaying too fast, use reduceLROnPlateau() with average validation loss
- use average of train and val loss for evaluation
- implement tensorboard logging

# Done
- trained model for 39 epochs on all shapes, good training error, val error is very high, might be a bug